### Project goal: Forecast the temperatures for Berlin-Tempelhof

Exercise in three notebooks:
1. Inspect the raw data and apply trend, seasonality and AR modelling manually and automatically using `statsmodels` package to predict temperature.
2. Use ARIMA and SARIMAX modelling to predict temperature.
3. Plot the predicted temperatures using Plotly

This is notebook 3.

In [1]:
# Imports
import json
import plotly.express as px
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim

We have successfully obtained some forecast for the next months in the previous notebook. Let's now plot it directly onto a map so that we can easily communicate this forecast. We use interactive plotly maps for that, which allows us to plot geoJSON files that contain a geometry shape for the area on a map. To that area, we plot the temperature forecast data in some meaningful and easy to interprete way.

We first read in geoJSON with geopandas, then load our predictions from the other notebook.

In [2]:
gpf = gpd.read_file("geo_data/bezirksgrenzen.geojson")
gpf = gpf.loc[gpf["Gemeinde_name"] == "Tempelhof-Schöneberg"][
    ["geometry", "Gemeinde_name"]
].reset_index(drop=True)
df = pd.read_pickle("prediction_future.pkl")

Add a date and area column, improve column names descriptives and et a df ready with temperature, area and year.  
Then merge into the geojson file (i.e., don't add geojson to the df). This will preserve the polygons as a geometry object.

In [3]:
df_plot = df["TG"]
df_plot_tempelhof_monthly = pd.DataFrame(df_plot.reset_index(drop=True))
df_plot_tempelhof_monthly["Date"] = pd.DataFrame(
    df_plot.index.year.astype(str) + " - " + df_plot.index.month_name()
)
df_plot_tempelhof_monthly["Gemeinde_name"] = gpf["Gemeinde_name"][0]
df_plot_tempelhof_monthly = gpf.merge(df_plot_tempelhof_monthly)
df_plot_tempelhof_monthly = df_plot_tempelhof_monthly.rename(
    {"TG": "Predicted temperature", "Gemeinde_name": "District name"}, axis=1
)

We can now dump this as a json file. We will store and reload the JSON file, not because we need to but just as an exercise.

In [4]:
tempelhof_monthly_json = df_plot_tempelhof_monthly.to_json()
with open("tempelhof_monthly_json.json", "w") as file:
    json.dump(tempelhof_monthly_json, file, indent=2)
with open("tempelhof_monthly_json.json", "r") as file:
    tempelhof_monthly_json = json.load(file)

Now, let's get the latitude and longitude of some point to center the map. I will obviously chose my Tennis club as center point which happens to be next to Berlin Tempelhof:

In [5]:
address = Nominatim(user_agent="mymap").geocode("Paradestraße 28-32, 12101 Berlin")
coordinates = {"lat": address.latitude, "lon": address.longitude}

We are now ready to plot the data using Plotly's choropleth.  
Choropleth is a method to generate an interactive map figure using plotly. The mapbox version of it creates animated features instead of just a static plot.

Before we plot use `json.loads()` on your loaded JSON file. This is a crucial step: this will make sure the geoJSON is loaded as a JSON dict rather than a JSON str which it normally is when loaded from a file using `json.load()`. JSON strings are accepted by plotly but won't read correctly and the methods do not throw an error or warning.

In [ ]:
tempelhof_monthly_json = json.loads(tempelhof_monthly_json)
data = df_plot_tempelhof_monthly.drop("geometry", axis=1)
# Now, let's actually plot but provide some explanation of the keyword arguments:
fig = px.choropleth_mapbox(
    data_frame=data,  # df that contains all years (can be gdf or df)
    hover_name="District name",  # text displayed when hovering over plot elements
    geojson=tempelhof_monthly_json,
    featureidkey="properties.District name",  # name of JSON key within the "properties" value that contains Gemeinde_name names
    animation_frame="Date",  # df columns over which the animation cycles
    locations="District name",  # name of the df column that contains Gemeinde_name names
    mapbox_style="open-street-map",  # see other styles: https://plotly.com/python/mapbox-layers/
    title="2022 Forecast for Berlin-Tempelhof (monthly average temperature)",
    color="Predicted temperature",  # name of the dataframe column that contains numerical data you want to display
    color_continuous_scale="RdBu_r",  # see other options: https://plotly.com/python/builtin-colorscales/
    range_color=(
        -10,
        30,
    ),  # range of color bar. is an iterative over df rows, so we have to provide it a fixed tuple of ints instead of defining it in dependence with the df
    opacity=0.6,  # opacity of the colored field
    center=coordinates,  # location the map will be center on at startup
    zoom=9,  # zoom setting at startup
)

# We could also update text elements by altering the font or style like so:
# fig.update_layout(
#     font_family="Times New Roman",
#     font_size=13,
#     font_color="black",
#     title_font_family="Times New Roman",
#     title_font_color="blue",
#     title_font_size=30,
# )

# We can add some sliders to manipulate the data
sliders = [
    dict(
        currentvalue={"font": {"size": 20}, "prefix": "Month: ", "xanchor": "right"},
        pad={"t": 0, "b": 30},
    )
]
# Some modifications on the slider
fig.update_layout(sliders=sliders)
fig.update_layout(margin={"r": 0, "t": 50, "l": 0, "b": 0})
fig["layout"]["updatemenus"][0]["buttons"][0]["args"][1]["frame"][
    "duration"
] = 1000  # Presentation duration of each data point when starting the animation
fig.layout.updatemenus[0]["pad"] = {"t": 100, "b": 20, "l": 20}
# We could also access it individually like so:
# fig.layout.updatemenus[0]["pad"]["t"] = 100
# fig.layout.updatemenus[0]["pad"]["b"] = 20
# fig.layout.updatemenus[0]["pad"]["l"] = 20
# Save it as HTML and show it
fig.write_html("tempelhof_temperature_monthly.html", include_plotlyjs="cdn")
fig.show()
# Note that this is HTML so we should be able to just copy and paste it into our blog.